In [1]:
from skr_web_api import Submission
import pandas as pd
import numpy as np
import regex as re
import argparse

In [2]:
df = pd.read_csv('discharge.csv', parse_dates=['charttime', 'storetime'])
df = df.drop(columns=['note_type', 'note_seq'])
df = df.dropna()
#df.groupby('subject_id').first()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 331777 entries, 0 to 331793
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   note_id     331777 non-null  object        
 1   subject_id  331777 non-null  int64         
 2   hadm_id     331777 non-null  int64         
 3   charttime   331777 non-null  datetime64[ns]
 4   storetime   331777 non-null  datetime64[ns]
 5   text        331777 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(2)
memory usage: 17.7+ MB


In [8]:
df = df.groupby('subject_id').first()
df = df.reset_index()

In [9]:
df

,subject_id,note_id,hadm_id,charttime,storetime,text
0,10000032,10000032-DS-21,22595853,2180-05-07,2180-05-09 15:26:00,\nName: ___ Unit No: _...
1,10000084,10000084-DS-17,23052089,2160-11-25,2160-11-25 15:09:00,\nName: ___ Unit No: __...
2,10000117,10000117-DS-21,22927623,2181-11-15,2181-11-15 15:04:00,\nName: ___ Unit No: ___\n...
3,10000248,10000248-DS-10,20600184,2192-11-30,2192-11-30 19:49:00,\nName: ___ Unit No: ...
4,10000560,10000560-DS-15,28979390,2189-10-17,2189-10-17 13:47:00,\nName: ___ Unit No: _...
...,...,...,...,...,...,...
145901,19999625,19999625-DS-12,27638769,2138-10-09,2138-10-10 01:42:00,\nName: ___ Unit No: ___\n \n...
145902,19999784,19999784-DS-10,25715748,2119-08-19,2119-08-19 10:09:00,\nName: ___ Unit No: __...
145903,19999828,19999828-DS-6,29734428,2147-08-04,2147-08-12 15:36:00,\nName: ___ Unit No: ___...
145904,19999840,19999840-DS-20,26071774,2164-07-28,2164-07-29 14:52:00,\nName: ___ Unit No: ___\...


In [10]:
corpus = df.text[1]
print(corpus)

 
Name:  ___                    Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: MEDICINE
 
Allergies: 
No Known Allergies / Adverse Drug Reactions
 
Attending: ___.
 
Chief Complaint:
Visual hallucinations
 
Major Surgical or Invasive Procedure:
N/A

 
History of Present Illness:
___ male with ___ disease, dyslipidemia, and a
history of prostate cancer (s/p prostatectomy) who was referred
to the ED by his neurologist for worsening gait, falls, and
visual hallucinations. 

The following history is taken from chart review: 

The patient was seen by his neurologist on ___ at which time he
was noted to have visual hallucinations and worsening gait
freezing. For his gait freezing, his mirapex was increased by
0.125 mg every week to a goal dose of 0.75 mg t.i.d. He
successfully up-titrated the medicine to 0.75/0.625/0.625 but
began to have visual hallucinations and confusion so on ___ his
neurologist recommended d

We are extracting present illness and hospital course:<br>
“History of Present Illness:” to “Past Medical History:”<br>
“Brief Hospital Course:” to “ Medications on Admission:”<br>
So maybe change the codes to<br>
match = re.search("History of Present Illness:(.*?)Past Medical History:", df['text'][0], re.DOTALL)<br>
match = re.search("Brief Hospital Course:(.*?)Medications on Admission:", df['text'][0], re.DOTALL)

In [11]:
def regex_match(text):
    extract = ''
    match1 = re.search("History of Present Illness:(.*?)Past Medical History:", text, re.DOTALL)
    match2 = re.search("Brief Hospital Course:(.*?)Medications on Admission:", text, re.DOTALL)
    if match1:
        extract = match1.group(1)
    if match2:
        extract += '\n\n'+ match2.group(1)
    else:
        extract = None
    with 
# Extracting content from "Brief Hospital Course" to "Discharge Medications" including newlines
regex_match(corpus)

AttributeError: 'NoneType' object has no attribute 'group'

In [7]:
if match1: 
    extract = match1.group(1).strip()
if match2:
    extract += '\n\n'+match2.group(1).strip()
else: 
    extract = None
extract
#save as txt file
print(extract)
with open('extract.txt', 'w') as f:
    f.write(extract)

___ with HIV on HAART, COPD, HCV cirrhosis complicated by 
ascites and HE admitted with abdominal distention and pain. She 
was admitted to ___ for the same symptoms 
recently and had 3L fluid removed (no SBP) three days ago and 
felt better. Since discharge, her abdomen has become 
increasingly distended with pain. This feels similar to prior 
episodes of ascites.  
Her diuretics were recently decreased on ___ due to worsening 
hyponatremia 128 and hyperkalemia 5.1. Patient states she has 
been compliant with her HIV and diuretic medications but never 
filled out the lactulose prescription. She states she has had 
___ BMs daily at home. She has had some visual hallucinations 
and forgetfulness. Her appetite has been poor.  
In the ED, initial vitals were 98.9 88 116/88 18 97% RA. CBC 
near baseline, INR 1.4, Na 125, Cr 0.6. AST and ALT mildly above 
baseline 182 and 126 and albumin 2.8. Diagnostic para with 225 
WBC, 7% PMN, total protein 0.3. UA with few bact, 6 WBC, mod 
leuk, neg n

In [11]:
email = 'dmd2225@cumc.columbia.edu'
apikey = '19895da9-5d2d-41da-80a3-26a6e3b55fdf'
input_file = 'extract.txt'
inst = Submission(email, apikey=apikey)
inst.set_mm_ksource('2022AA')
inst.init_mm_interactive(extract, ksource='2022AA', args='-I --negex --conj')
response = inst.submit()
print(f'response status: {response.status_code}')
print(f'content:\n {response.content.decode()}')

response status: 200
content:
 /dmzfiler/II_Group/MetaMap2020/public_mm/bin/SKRrun.20 /dmzfiler/II_Group/MetaMap2020/public_mm/bin/metamap20.BINARY.Linux --lexicon db -Z 2022AA --silent -I --negex
Processing USER.tx.1: ___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD,  bioplar, PTSD, presented from OSH ED with worsening abd  distension over past week.   

Phrase: ___ HCV cirrhosis c/b ascites,
Meta Mapping (762):
   637   C0019196:HCV (Hepatitis C) [Disease or Syndrome]
   637   C0023890:CIRRHOSIS (Liver Cirrhosis) [Disease or Syndrome]
   804   C0003962:ASCITES (Ascites) [Disease or Syndrome]
Meta Mapping (762):
   637   C0019196:HCV (Hepatitis C) [Disease or Syndrome]
   637   C0023890:CIRRHOSIS (Liver Cirrhosis) [Disease or Syndrome]
   804   C4553641:Ascites (Ascites, CTCAE) [Finding]
Meta Mapping (762):
   637   C0019196:HCV (Hepatitis C) [Disease or Syndrome]
   637   C1623038:Cirrhosis [Disease or Syndrome]
   804   C0003962:ASCITES (Ascites) [Disease or Syndrome]
Meta 

In [10]:
email = 'dmd2225@cumc.columbia.edu'
apikey = '19895da9-5d2d-41da-80a3-26a6e3b55fdf'
input_file = 'extract.txt'
inst = Submission(email, apikey=apikey)
inst.set_mm_ksource('2022AA')
inst.init_mm_interactive(extract)
response = inst.submit()
print(f'response status: {response.status_code}')
print(f'content:\n {response.content.decode()}')

response status: 200
content:
 /dmzfiler/II_Group/MetaMap2020/public_mm/bin/SKRrun.20 /dmzfiler/II_Group/MetaMap2020/public_mm/bin/metamap20.BINARY.Linux --lexicon db -Z 2020AB --silent -N
USER|MMI|33.49|Deny (action)|C2700401|[acty]|["Deny"-tx-13-"denies"-verb-0,"Deny"-tx-12-"denies"-verb-0,"Deny"-tx-9-"denies"-verb-0,"Deny"-tx-8-"denies"-verb-0,"Deny"-tx-6-"denies"-verb-0,"Deny"-tx-5-"denies"-verb-0]|TX|1132/6;1089/6;811/6;667/6;509/6;465/6|
USER|MMI|14.88|Endocrine System Findings Domain|C3538926|[inpr]|["ED"-tx-14-"ED"-noun-0,"ED"-tx-11-"ED"-noun-0,"ED"-tx-10-"ED"-noun-0,"ED"-tx-1-"ED"-noun-0]|TX|1211/2;982/2;929/2;94/2|
USER|MMI|11.92|Regulatory Report|C3273238|[inpr]|["Report"-tx-13-"reports"-verb-0,"Report"-tx-11-"report"-noun-0,"Report"-tx-2-"reports"-verb-0]|TX|1162/7;985/6;149/7|
USER|MMI|11.92|Report (document)|C0684224|[inpr]|["Reports"-tx-13-"reports"-verb-0,"Report"-tx-11-"report"-noun-0,"Reports"-tx-2-"reports"-verb-0]|TX|1162/7;985/6;149/7|
USER|MMI|11.92|Report:Finding